Getting and reading CCSDS OEM ephemerides for the ISS using Orekit.

In [1]:
import orekit
if orekit.getVMEnv() is None:
    orekit.initVM()

import os
from orekit.pyhelpers import download_orekit_data_curdir, setup_orekit_curdir
if not os.path.exists('orekit-data.zip'):
    download_orekit_data_curdir()
setup_orekit_curdir()

In [2]:
from datetime import datetime, timedelta
date = datetime(2021, 4, 2, 17, 4, 29)
duration_s = 3600.0  # seconds, minimum duration starting from date that we want to have in the OEM file

In [3]:
directory_before_2021 = 'ISS.OEM_J2K_EPH'
directory_from_2021 = 'ISS_OEM'

Searching for the best OEM file for the given date. This part only works from October 2020. If ephemeris files are already available on the local computer, then this block is not needed anymore, just use `oem_file = OEMParser().parse(filename)`.

In [4]:
import urllib.request
from orekit.pyhelpers import absolutedate_to_datetime, datetime_to_absolutedate
from org.orekit.files.ccsds import OEMParser

date_to_try = date
error = True

while error:
    try:
        filename = f'ISS.OEM_J2K_EPH_{date_to_try:%Y-%m-%d}.txt'

        if date_to_try.year < 2021:
            directory = directory_before_2021
        else:
            directory = directory_from_2021

        # Downloading CCSDS OEM file        
        target_url = f'https://nasa-public-data.s3.amazonaws.com/iss-coords/{date_to_try:%Y-%m-%d}/{directory}/ISS.OEM_J2K_EPH.txt'
        urllib.request.urlretrieve(target_url, filename)
        print(f'Successfully downloaded CCSDS OEM for date {date_to_try:%Y-%m-%d}')

        # Replacing wrong keyword USABLE_START_TIME to USEABLE_START_TIME in OEM files prior to August 2021...
        with open(filename) as f:
            newText=f.read().replace('USABLE', 'USEABLE')
        with open(filename, "w") as f:
            f.write(newText)

        # Parsing the OEM file using Orekit
        oem_file = OEMParser().parse(filename)
        # Reading the first ephemeris block in the CCSDS OEM file. 
        # We assume that the ISS OEMs always contain only one block, which seems to be always the case.
        ephem_first_block = oem_file.getEphemeridesBlocks().get(0)

        # Checking if the OEM file contains data for the desired time range
        if absolutedate_to_datetime(ephem_first_block.getStartTime()) > date:
            print('The current OEM file starts after the desired date, searching another file at an earlier date')
            date_to_try = date_to_try + timedelta(days=-1)
        elif absolutedate_to_datetime(ephem_first_block.getStopTime()) < (date + timedelta(seconds=duration_s)):
            print('Something is wrong, the OEM file (normally 15 days long) stops before the desired time range')      
            break
        else:
            print('The current OEM file contains data at the date we want')
            error = False
        
    except:
        print(f'Date {date_to_try:%Y-%m-%d} has no available data, retrying an earlier date')
        date_to_try = date_to_try + timedelta(days=-1)

Date 2021-04-02 has no available data, retrying an earlier date
Successfully downloaded CCSDS OEM for date 2021-04-01
The current OEM file contains data at the date we want


In [5]:
from org.orekit.files.general import EphemerisFile
from orekit.pyhelpers import datetime_to_absolutedate
# Must cast to the EphemerisFile.EphemerisSegment interface because of the limitations of the Orekit Python wrapper
bounded_propagator = EphemerisFile.EphemerisSegment.cast_(ephem_first_block).getPropagator()
date_min = bounded_propagator.getMinDate()
date_max = bounded_propagator.getMaxDate()
date_end = date + timedelta(seconds=86400)
print(f'File contains data from {date_min} to {date_max}')
print(f'We will process data from {datetime_to_absolutedate(date)} to {datetime_to_absolutedate(date_end)}')

File contains data from 2021-03-31T12:00:00.000 to 2021-04-15T12:00:00.000
We will process data from 2021-04-02T17:04:29.000 to 2021-04-03T17:04:29.000


In [6]:
bounded_propagator.getFrame()

<Frame: EME2000>

In [7]:
from org.orekit.frames import FramesFactory, ITRFVersion
from org.orekit.utils import IERSConventions
tod = FramesFactory.getTOD(IERSConventions.IERS_2010, False) # Taking tidal effects into account when interpolating EOP parameters
eme2000 = FramesFactory.getEME2000()
gcrf = FramesFactory.getGCRF()
itrf = FramesFactory.getITRF(IERSConventions.IERS_2010, False)

from org.orekit.models.earth import ReferenceEllipsoid
wgs84Ellipsoid = ReferenceEllipsoid.getWgs84(itrf)

from org.orekit.time import TimeScalesFactory
utc = TimeScalesFactory.getUTC()

In [8]:
import pandas as pd
ground_stations_file = 'ground-stations.csv'
ground_station_df = pd.read_csv(ground_stations_file, index_col=0)

display(ground_station_df)

,longitude_deg,latitude_deg,altitude
name,,,
Svalbard,15.0,78.0,0.0
Australia,117.0,-32.0,0.0
Argentina,-65.0,-35.0,0.0
Antarctica,-75.0,-73.0,0.0
Tunka,102.5,51.7,0.0


In [9]:
import numpy as np

sigma_position = 1.0  # Position noise in meters
sigma_velocity = 1e-3 # Velocity noise in meters per second
step_pv = 60.0  # Step time in seconds for output PV data

sigma_range = 1.0  # Range noise in meters
sigma_range_rate = 1e-3  # Range rate noise in meters per second
sigma_az = float(np.deg2rad(0.01))  # Azimuth noise in radians
sigma_el = float(np.deg2rad(0.01))  # Elevation noise in radians
sigma_ra = float(np.deg2rad(0.01))  # Right ascension noise in radians
sigma_dec = float(np.deg2rad(0.01))  # Declination noise in radians
step_ground_station_measurements = 10.0  # When a ground station is in view, take measurements every 10 seconds

In [10]:
from org.orekit.bodies import GeodeticPoint
from org.orekit.frames import TopocentricFrame
from org.orekit.propagation.events import ElevationDetector
from org.orekit.estimation.measurements import GroundStation, Range, AngularAzEl, ObservableSatellite
from org.orekit.estimation.measurements.generation import PVBuilder, RangeBuilder, RangeRateBuilder, AngularAzElBuilder, AngularRaDecBuilder, EventBasedScheduler, SignSemantic, Generator, ContinuousScheduler
from org.orekit.time import FixedStepSelector
from org.orekit.estimation.measurements import ObservableSatellite
from org.orekit.propagation.events.handlers import ContinueOnEvent

observableSatellite = ObservableSatellite(0) # Propagator index = 0

meas_generator = Generator()
meas_generator.addPropagator(bounded_propagator)

# Add PV builder
meas_generator.addScheduler(
    ContinuousScheduler(
        PVBuilder(None,  # no noise
                  sigma_position,
                  sigma_velocity,
                  1.0,  # Base weight
                  observableSatellite), 
        FixedStepSelector(step_pv, utc))
)

for gs_name, gs_data in ground_station_df.iterrows():
    geodetic_point = GeodeticPoint(float(np.deg2rad(gs_data['latitude_deg'])),
                                   float(np.deg2rad(gs_data['longitude_deg'])),
                                   float(gs_data['altitude']))
    topocentric_frame = TopocentricFrame(wgs84Ellipsoid, geodetic_point, gs_name)
    ground_station_df.loc[gs_name, 'GroundStation'] = GroundStation(topocentric_frame)

    # Range builder
    meas_generator.addScheduler(
        EventBasedScheduler(
            RangeBuilder(None, 
                         GroundStation(topocentric_frame), 
                         False,  # one-way, this is important for telescope observations
                         sigma_range, 
                         1.0,  # Base weight
                         observableSatellite), 
            FixedStepSelector(step_ground_station_measurements, utc), 
            bounded_propagator, 
            ElevationDetector(topocentric_frame).withHandler(ContinueOnEvent()), 
            SignSemantic.FEASIBLE_MEASUREMENT_WHEN_POSITIVE)
    )

    # Range rate builder
    meas_generator.addScheduler(
        EventBasedScheduler(
            RangeRateBuilder(None,  # no noise
                             GroundStation(topocentric_frame), 
                             True,  # two-way
                             sigma_range_rate, 
                             1.0,  # Base weight
                             observableSatellite), 
            FixedStepSelector(step_ground_station_measurements, utc), 
            bounded_propagator, 
            ElevationDetector(topocentric_frame).withHandler(ContinueOnEvent()), 
            SignSemantic.FEASIBLE_MEASUREMENT_WHEN_POSITIVE)
    )

    # Az/el builder
    meas_generator.addScheduler(
        EventBasedScheduler(
            AngularAzElBuilder(None,  # no noise
                               GroundStation(topocentric_frame),
                               [sigma_az, sigma_el], 
                               [1.0, 1.0],  # Base weight
                               observableSatellite), 
            FixedStepSelector(step_ground_station_measurements, utc), 
            bounded_propagator, 
            ElevationDetector(topocentric_frame).withHandler(ContinueOnEvent()), 
            SignSemantic.FEASIBLE_MEASUREMENT_WHEN_POSITIVE)
    )

    # RA/DEC builder
    meas_generator.addScheduler(
        EventBasedScheduler(
            AngularRaDecBuilder(None,  # no noise
                               GroundStation(topocentric_frame),
                               eme2000,  # RA/DEC measurements are defined in Earth-centered inertial frame
                               [sigma_ra, sigma_dec], 
                               [1.0, 1.0],  # Base weight
                               observableSatellite), 
            FixedStepSelector(step_ground_station_measurements, utc), 
            bounded_propagator, 
            ElevationDetector(topocentric_frame).withHandler(ContinueOnEvent()), 
            SignSemantic.FEASIBLE_MEASUREMENT_WHEN_POSITIVE)
    )

Propagating, retrieving the generated measurements.

Warning: the cell below cannot be run a second time without running the cell above again before. Otherwise the result structure will be empty.

In [11]:
from orekit.pyhelpers import absolutedate_to_datetime
from org.orekit.estimation.measurements import ObservedMeasurement, PV, Range, RangeRate, AngularAzEl, AngularRaDec
from org.orekit.utils import PVCoordinates
from org.hipparchus.geometry.euclidean.threed import Vector3D

generated = meas_generator.generate(datetime_to_absolutedate(date), datetime_to_absolutedate(date_end))
pv_itrf_df = pd.DataFrame(columns=['x', 'y', 'z', 'vx', 'vy', 'vz'])
pv_eme2000_df = pd.DataFrame(columns=['x', 'y', 'z', 'vx', 'vy', 'vz'])
range_df = pd.DataFrame(columns=['ground_station', 'range'])
range_rate_df = pd.DataFrame(columns=['ground_station', 'range_rate'])
az_el_df = pd.DataFrame(columns=['ground_station', 'az_deg', 'el_deg'])
ra_dec_df = pd.DataFrame(columns=['ground_station', 'ra_deg', 'dec_deg'])

for meas in generated.toArray():
    observed_meas = ObservedMeasurement.cast_(meas)
    py_datetime = absolutedate_to_datetime(observed_meas.date)

    if PV.instance_(observed_meas):
        '''
        PV objects are given in propagator frame (ECI)
        We transform them to ITRF and to EME2000 frame (depending on the user's needs)
        '''
        observed_pv = PV.cast_(observed_meas)
        pv_eci_jarray = observed_meas.getObservedValue()
        pv_eci = PVCoordinates(Vector3D(pv_eci_jarray[0:3]), Vector3D(pv_eci_jarray[3:6]))
        
        eci_to_itrf = eme2000.getTransformTo(itrf, observed_meas.date)
        pv_itrf = eci_to_itrf.transformPVCoordinates(pv_eci)
        pv_itrf_df.loc[py_datetime] = np.concatenate(([np.array(pv_itrf.getPosition().toArray()),
                                                       np.array(pv_itrf.getVelocity().toArray())]))
        
        pv_eme2000_df.loc[py_datetime] = np.concatenate(([np.array(pv_eci.getPosition().toArray()),
                                                       np.array(pv_eci.getVelocity().toArray())]))
        
        

    elif Range.instance_(observed_meas):
        observed_range = Range.cast_(observed_meas)
        range_df.loc[py_datetime] = np.concatenate(([observed_range.getStation().getBaseFrame().name], 
                                                    np.array(observed_range.getObservedValue())))

    elif RangeRate.instance_(observed_meas):
        observed_range_rate = RangeRate.cast_(observed_meas)
        range_rate_df.loc[py_datetime] = np.concatenate(([observed_range_rate.getStation().getBaseFrame().name], 
                                                         np.array(observed_range_rate.getObservedValue())))

    elif AngularAzEl.instance_(observed_meas):
        observed_az_el = AngularAzEl.cast_(observed_meas)
        az_el_df.loc[py_datetime] = np.concatenate(([observed_az_el.getStation().getBaseFrame().name], 
                                                    np.rad2deg(observed_az_el.getObservedValue())))

    elif AngularRaDec.instance_(observed_meas):
        observed_ra_dec = AngularRaDec.cast_(observed_meas)
        ra_dec_df.loc[py_datetime] = np.concatenate(([observed_ra_dec.getStation().getBaseFrame().name], 
                                                     np.rad2deg(observed_ra_dec.getObservedValue())))

Position and velocity in ITRF frame

In [12]:
pv_itrf_df

,x,y,z,vx,vy,vz
2021-04-02 17:05:00,-4.182145e+06,8.009900e+05,-5.304967e+06,-622.061435,-7289.313639,-611.931515
2021-04-02 17:06:00,-4.211910e+06,3.622922e+05,-5.329586e+06,-369.963199,-7328.792269,-208.591411
2021-04-02 17:07:00,-4.226516e+06,-7.784675e+04,-5.329985e+06,-116.652515,-7337.354213,195.381393
2021-04-02 17:08:00,-4.225873e+06,-5.175725e+05,-5.306127e+06,138.443045,-7315.026764,599.986897
2021-04-02 17:09:00,-4.209891e+06,-9.550236e+05,-5.258019e+06,393.836227,-7261.568664,1002.593041
...,...,...,...,...,...,...
2021-04-03 17:00:00,3.459316e+06,-3.345232e+06,4.794761e+06,2546.919973,6387.726654,2610.406578
2021-04-03 17:01:00,3.605813e+06,-2.955274e+06,4.940241e+06,2335.049361,6605.965113,2237.459831
2021-04-03 17:02:00,3.739375e+06,-2.553100e+06,5.063092e+06,2115.747288,6794.970835,1856.192716
2021-04-03 17:03:00,3.859545e+06,-2.140458e+06,5.162820e+06,1888.571342,6954.936885,1466.719773


Position and velocity in EME2000 frame

In [13]:
pv_eme2000_df

,x,y,z,vx,vy,vz
2021-04-02 17:05:00,-1.012121e+06,-4.138797e+06,-5.302883e+06,7551.469164,-1045.207151,-627.254702
2021-04-02 17:06:00,-5.570789e+05,-4.192073e+06,-5.328426e+06,7610.829717,-730.154291,-224.037350
2021-04-02 17:07:00,-9.951223e+04,-4.226354e+06,-5.329754e+06,7635.632954,-412.036746,179.882841
2021-04-02 17:08:00,3.585059e+05,-4.241456e+06,-5.306826e+06,7625.878875,-90.854515,584.505871
2021-04-02 17:09:00,8.148917e+05,-4.237230e+06,-5.259644e+06,7581.226752,231.341666,987.200388
...,...,...,...,...,...,...
2021-04-03 17:00:00,3.532944e+06,3.277911e+06,4.787565e+06,-6478.052767,3137.575234,2623.537372
2021-04-03 17:01:00,3.136459e+06,3.458506e+06,4.933849e+06,-6732.765610,2880.236936,2251.109631
2021-04-03 17:02:00,2.725650e+06,3.623305e+06,5.057532e+06,-6955.583120,2611.097087,1870.296842
2021-04-03 17:03:00,2.302427e+06,3.771604e+06,5.158119e+06,-7146.537474,2330.252195,1481.213611


Azimuth and elevation, here selecting only the Tunka ground station.

In [14]:
az_el_df[az_el_df['ground_station'] == 'Tunka']

,ground_station,az_deg,el_deg
2021-04-03 06:03:50,Tunka,-172.79327750314087,0.16317128247147406
2021-04-03 06:04:00,Tunka,-173.95739849771678,0.6515985379198606
2021-04-03 06:04:10,Tunka,-175.17759402370638,1.1431361365672013
2021-04-03 06:04:20,Tunka,-176.45699441069658,1.6372880423105527
2021-04-03 06:04:30,Tunka,-177.79882048807946,2.1334725880160113
...,...,...,...
2021-04-03 14:12:30,Tunka,-159.9832725632749,1.4415464895889194
2021-04-03 14:12:40,Tunka,-161.62965939596506,1.1067518881130827
2021-04-03 14:12:50,Tunka,-163.22352335845432,0.7621785076861289
2021-04-03 14:13:00,Tunka,-164.765223298076,0.40908851515635786


Right ascension and declination when the spacecraft is in view of any station. Although the RA/DEC coordinates are in theory independent on the station's position, here the time-of-flight to a ground telescope for instance plays a role.

In [15]:
ra_dec_df

,ground_station,ra_deg,dec_deg
2021-04-02 18:26:00,Australia,136.01623905133476,1.8957012638504989
2021-04-02 18:26:10,Australia,135.97928167492086,0.7595390059760352
2021-04-02 18:26:20,Australia,135.9229171443263,-0.42394491379683996
2021-04-02 18:26:30,Australia,135.84560991044393,-1.657743763711984
2021-04-02 18:26:40,Australia,135.74564303284095,-2.945020636219737
...,...,...,...
2021-04-03 16:10:50,Australia,-63.943566215111595,-27.305157302499616
2021-04-03 16:11:00,Australia,-62.85976032076412,-27.47124245999542
2021-04-03 16:11:10,Australia,-61.81809629442549,-27.607617182678684
2021-04-03 16:11:20,Australia,-60.815753380840476,-27.717013066548525
